In [1]:
from datasets.download import DownloadManager
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss

MODEL_NAME = "intfloat/multilingual-e5-small"

/Users/hotchpotch/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# データセットのロード
wikija_dataset = load_dataset(
    path="singletongue/wikipedia-utils",
    name="passages-c400-jawiki-20230403",
    split="train",
)
# faiss index のダウンロード
dm = DownloadManager()
index_local_path = dm.download(
    f"https://huggingface.co/datasets/hotchpotch/wikipedia-passages-jawiki-embeddings/resolve/main/faiss_indexes/passages-c400-jawiki-20230403/multilingual-e5-small-passage/index_IVF2048_PQ96.faiss"
)
# faiss index のロード
faiss_index = faiss.read_index(index_local_path)

In [3]:
# embeddings へ変換するモデルのロード

model = SentenceTransformer("intfloat/multilingual-e5-small")
model.max_seq_length = 512

In [4]:
def to_emb(model, text, prefix="query: "):
    return model.encode([prefix + text], normalize_embeddings=True)


emb = to_emb(model, "1975年に『アザミ嬢のララバイ』でデビューした女性歌手で、『わかれうた』『地上の星』などの曲を出しているのは誰?")

In [5]:
TOP_K = 5
scores, indexes = faiss_index.search(emb, TOP_K)
for idx, (id, score) in enumerate(zip(indexes[0], scores[0])):
    data = wikija_dataset[int(id)]
    print((score, data["title"], data["text"][:100]))

(0.21511322, 'アザミ嬢のララバイ', '「アザミ嬢のララバイ」(アザミじょうのララバイ)は、中島みゆきのデビュー・シングル。1975年9月25日にキャニオン・レコードよりリリース(レーベルはAARD-VARK)。規格品番:AV-69。')
(0.24348262, 'わかれうた', '今作で、自身初のオリコンシングルチャート1位を獲得した。この後中島は、オリコンにおいて4つの年代に渡ってシングルチャート1位を獲得することになる。カップリングの「ホームにて」が、JR東日本の特別企画乗')
(0.24827655, 'アザミ嬢のララバイ', '3⁄4拍子のワルツである。1stアルバム『私の声が聞こえますか』にはアレンジ違いのアルバムバージョンが収録された。本作発表・発売前の1972年に「フォーク音楽祭全国大会」に出場し「あたし時々おもうの」')
(0.2785422, 'わかれうた', '「わかれうた」は、中島みゆきの楽曲で、5枚目のシングル。1977年9月10日にキャニオン・レコードよりリリース。累計売上はミリオンセラーに達した。')
(0.2917346, '地上の星/ヘッドライト・テールライト', '「地上の星/ヘッドライト・テールライト」(ちじょうのほし / ヘッドライト・テールライト)は、中島みゆきの37作目のシングル。2000年7月19日にヤマハミュージックコミュニケーションズから発売された')
